In [39]:
.engine.model.instrument.Instrument:([iId:`long$()] cntTyp:`long$();state:`long$();faceValue:`long$();mkprice:`long$();
   plmts:`long$();plmtb:`long$();lotSize:`long$();tickSize:`long$();mnSize:`long$();
   mxSize:`long$();mnPrice:`long$();mxPrice:`long$();smul:`long$();
   bestBid:`long$();bestAsk:`long$();liqb:`boolean$();liqs:`boolean$());

In [34]:
.engine.model.inventory.Inventory:([aId:`long$();side:`long$()] iId:`.engine.model.instrument.Instrument$();
   ordQty:`long$();ordVal:`long$();ordLoss:`long$();amt:`long$();iw:`long$();mm:`long$();
   posVal:`long$();rpnl:`long$();avgPrice:`long$();execCost:`long$();upnl:`long$();lev:`long$());

In [5]:
\l test.q

"###################################################################################################"
"                                             TEST"
"###################################################################################################"
"==================================================================================================="
"35) .engine.logic.orderbook.Level (UNIT) :READY"
`Given a depth update which consists of a table of time,side,pricesize update the orderbook and the individual order offsets/icebergorders where necessary
"---------------------------------------------------------------------------------------------------"
caseId state dscr                                            
-------------------------------------------------------------
262    PASS  No change occurs and thus no update is triggered
"                                                                                                                                                    

In [6]:
.util.testutils.makeDefaultsRecords  :{[tab;cl;vl] // TODO inter with actual cols
    r:.util.NullRowDict[tab];
    cvl:count[vl]; 
    :$[cvl>1;[rx:(cvl#enlist[r]);rx[cl]:flip[vl];:rx];[r[cl]:first[vl];:r]]};

In [7]:
.util.testutils.makeInstrument     :{[cl;vl]
    .util.testutils.makeDefaultsRecords[`.engine.model.instrument.Instrument;cl;vl]
    };

In [12]:
// Checks that the .engine.model.order.Order table matches the orders
// that are provided.
/  @param x (Order/List) The orders that are to be checked
/  @param y (Case) The case that the assertions belong to
/  @param z (List[String]) The params that are being checked 
.util.testutils.makeInventory     :{[cl;vl]
    .util.testutils.makeDefaultsRecords[`.engine.model.inventory.Inventory;cl;vl]
    };

In [9]:
.engine.model.instrument.Instrument,:.util.testutils.makeInstrument[`mkprice`tickSize`mnSize;enlist(1000;5;1)]

In [18]:
aIds:(0 1);

In [22]:
.engine.model.risktier.RiskTier,:flip `rtid`mxamt`mmr`imr`maxlev!flip[(
        (0; 50000;       0.004;    0.008;    125);
        (1; 250000;      0.005;    0.01;     100);
        (2; 1000000;     0.01;     0.02;     50);
        (3; 5000000;     0.025;    0.05;     20);
        (4; 20000000;    0.05;     0.1;      10);
        (5; 50000000;    0.1;      0.20;     5);
        (6; 100000000;   0.125;    0.25;     4);
        (7; 200000000;   0.15;     0.333;    3);
        (8; 500000000;   0.25;     0.50;     2);
        (9; 500000000;   0.25;     1.0;      1))]; 

     .engine.model.feetier.FeeTier,:flip `ftid`vol`mkrfee`tkrfee`wdrawfee`dpstfee`wdlim!flip[(
        (0; 50;      0.0006;    0.0006;    0f;  0f; 600);
        (1; 500;     0.00054;   0.0006;    0f;  0f; 600);
        (2; 1500;    0.00048;   0.0006;    0f;  0f; 600);
        (3; 4500;    0.00042;   0.0006;    0f;  0f; 600);
        (4; 10000;   0.00042;   0.00054;   0f;  0f; 600);
        (5; 20000;   0.00036;   0.00048;   0f;  0f; 600);
        (6; 40000;   0.00024;   0.00036;   0f;  0f; 600);
        (7; 80000;   0.00018;   0.000300;  0f;  0f; 600);
        (8; 150000;  0.00012;   0.00024;   0f;  0f; 600))];                             //  

In [40]:
.engine.model.instrument.Instrument,:((!) . flip(
        (`iId                      ; 0);                                           
        (`cntTyp                   ; 0);
        (`state                    ; 0);
        (`faceValue                ; 0);
        (`mkprice                  ; 0);
        (`mxSize                   ; 0);
        (`mnPrice                  ; 0);
        (`smul                     ; 0);
        (`mxPrice                  ; 0) 
        ));

In [36]:
n:count[aIds];
dn:n*2;
.engine.model.inventory.Inventory,:ivn:flip[(!) . flip(
    (`aId              ; floor[til[dn]%2]); 
    (`iId              ; `.engine.model.instrument.Instrument$(dn#0)); 
    (`side             ; dn#(1 -1));                                            
    (`ordQty           ; dn#0);
    (`ordVal           ; dn#0);
    (`ordLoss          ; dn#0);
    (`amt              ; dn#0);
    (`iw               ; dn#0); // isolated wallet
    (`mm               ; dn#0);
    (`posVal           ; dn#0);
    (`rpnl             ; dn#0);  
    (`avgPrice         ; dn#0);  
    (`execCost         ; dn#0);  
    (`upnl             ; dn#0);  
    (`lev              ; dn#0)  
    )];

In [24]:
.engine.model.account.Account,:acc:flip((!) . flip(
        (`aId              ; aIds);                                            
        (`lng              ; `.engine.model.inventory.Inventory$(flip(aIds;n#1)));
        (`srt              ; `.engine.model.inventory.Inventory$(flip(aIds;n#-1)));
        (`dep              ; n#0);  
        (`wit              ; n#0);  
        (`rt               ; n#`.engine.model.risktier.RiskTier$0);  
        (`ft               ; n#`.engine.model.feetier.FeeTier$0);  
        (`posTyp           ; n#0);  
        (`mrgTyp           ; n#0);  
        (`avail            ; n?10);  
        (`bal              ; n?10)  
        ));

In [25]:
update 
 mkprice:1000
 from `.engine.model.instrument.Instrument
 where iId=0;

In [47]:
update 
 upl:.engine.logic.contract.UnrealizedPnl[
     iId.cntTyp,
     iId.mkprice,
     iId.faceValue,
     iId.smul,
     amt,
     side,
     avgPrice]
    from `.engine.model.inventory.Inventory 
    where (iId=0) and (amt>0)

`.engine.model.inventory.Inventory


In [ ]:
// Apply mark price update 
.engine.logic.instrument.MarkPrice:{
				update 
				  	mkprice:x[`markprice] 
				  	from `.engine.model.instrument.Instrument
						where iId=x[`iId];

				update 
					  upl:.engine.logic.contract.UnrealizedPnl[
								iId.cntTyp,	
								iId.mkprice,
								iId.faceValue,
								iId.smul,
								amt,
								side,
								avgPrice]
						from `.engine.model.inventory.Inventory 
						where (iId=x`iId) and (amt>0);

				update
					ft:.engine.logic.account.GetFeetier[vol],
					rt:.engine.logic.account.GetRisktier[lng.amt+srt.amt],
					avail:.engine.logic.account.GetAvailable[
							bal;lng.mm+srt.mm;lng.upnl+srt.upnl;
							lng.ordQty+srt.ordQty;lng.ordLoss+srt.ordLoss]
					from `.engine.model.account.Account 
					where iId=x[`iId] and ((lng.amt>0) or (srt.amt>0));

				.engine.EmitA[`account;last t;last x];
				.engine.EmitA[`inventory;last t;last x];
				.engine.Emit[`mark;last t;last x];
	};
